In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone(api_keys=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '039b528aa4836f3b79cb9ac27e657d93', 'Date': 'Sun, 01 Sep 2024 12:55:53 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
import json
data=json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent professor! His lectures are very clear and engaging. He makes complex topics easy to understand.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Very knowledgeable and passionate about the subject. Sometimes goes a bit too fast, but overall great.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Good professor, but his lectures can be a bit dry. Needs more interactive sessions.'},
 {'professor': 'Dr. Lisa Davis',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Not very organized and often seems unprepared. However, she is helpful during office hours.'},
 {'professor': 'Dr. Michael Wilson',
  'subject': 'History',
  'stars': 4,
  'review': 'Great storytelling skills and makes history interesting. However, his grading can be quite tough.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Biology',
 

In [6]:
import json
from sentence_transformers import SentenceTransformer

# Load the data from reviews.json
with open('reviews.json', 'r') as file:
    data = json.load(file)

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract review texts
reviews = [review['review'] for review in data['reviews']]

# Generate embeddings
embeddings = model.encode(reviews)

# Process the embeddings as needed
processed_data = []
for i, review in enumerate(data['reviews']):
    processed_data.append({
        "values": embeddings[i].tolist(),  # Convert numpy array to list if needed
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

print(processed_data)


c:\Users\KIIT\rag\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'values': [-0.019153138622641563, -0.03105400688946247, -0.03966066241264343, -0.013521911576390266, -0.08408739417791367, -0.028803963214159012, 0.0007162695983424783, 0.023153938353061676, -0.054014500230550766, -0.01766170561313629, -0.021228380501270294, 0.08560451865196228, -0.0486430749297142, 0.048858433961868286, -0.0007468164549209177, -0.009769605472683907, -0.024221321567893028, -0.0800296738743782, -0.0007978382636792958, -0.06754519045352936, 0.05265996977686882, 0.06278584152460098, 0.07322800159454346, -0.0606500618159771, -0.021908005699515343, -0.04432285577058792, -0.0019028944661840796, -0.023586634546518326, 0.11396607011556625, -0.048161089420318604, -0.062305521219968796, 0.06910067051649094, 0.011311118490993977, 0.012839815579354763, -0.007535287644714117, 0.03163360431790352, 0.00020768992544617504, 0.08436034619808197, 0.006756118964403868, 0.03338596224784851, -0.02193286269903183, 0.06983523815870285, 0.042083755135536194, 0.0017888324800878763, -0.0117034

In [11]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 01 Sep 2024 12:56:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1382', 'x-pinecone-request-id': '4096860553016496803', 'x-envoy-upstream-service-time': '43', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 2048","details":[]}


In [13]:
# Delete the existing index
pc.delete_index('rag')



In [14]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Example: Create a new index with the correct dimension
pc.create_index(
    name="rag",
    dimension=384,  # Set to the dimension of your vectors
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [15]:
index = pc.Index('rag')

# Prepare your data for upsert
# Ensure `processed_data` is a list of dictionaries with `id` and `values` (vector)
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)


{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}